In [1]:
import pandas as pd
import yaml
import sys
import os
from glob import glob
from pathlib import Path
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')

sys.path.append('/kaggle/src')
from utils.xgb import fit_xgb
from utils.metric import compute_comptetition_metric
from utils.postprocess import post_process
from utils.set_seed import seed_base
from feature_engineering.ranker import generate_ranker_features

PACKAGE_DIR = Path("/kaggle/src")
CFG = yaml.safe_load(open(PACKAGE_DIR / "config.yaml", "r"))
print(CFG["ranker"]["execution"]["exp_id"])

CFG["output_dir"] = f"/kaggle/output/{CFG['ranker']['execution']['exp_id']}"
seed_base(CFG["env"]["seed"])

2023-10-06 03:17:25.206933: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-06 03:17:25.584126: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-06 03:17:26.723237: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/u

exp_066


In [2]:
sub = pd.read_csv(os.path.join(CFG["output_dir"], "submission.csv"))
sub.head()

,score,key_step,step,event,level,series_id
0,0.072770,4331.5,4275,onset,13.0,038441c925bb
1,0.745010,4691.5,4635,onset,11.0,038441c925bb
2,2.544872,4811.5,4755,onset,9.0,038441c925bb
3,5.886312,4931.5,4875,onset,7.0,038441c925bb
4,7.134280,4979.5,4923,onset,5.0,038441c925bb


In [4]:
import scipy
score2range = scipy.interpolate.interp1d([-100, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 100], [0, 0, 12, 36, 60, 90, 120, 150, 180, 240, 300, 360, 360])
range2score = scipy.interpolate.interp1d([0, 12, 36, 60, 90, 120, 150, 180, 240, 300, 360], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
score2range(1.5)

array(24.)

In [38]:
dfs = []
for _, df in tqdm(sub.groupby("series_id")):
    df = df.sort_values("score", ascending=False).reset_index(drop=True)
    used = np.zeros(len(df))
    reduce_rate = np.ones(df["step"].max() + 500)
    for _ in range(len(df)):
        best_score = -1e10
        best_idx = -1
        best_step = -1
        best_row = -1
        for i, row in df.iterrows():
            if used[i]:
                continue
            score = row["score"] / reduce_rate[row["step"]]
            if score > best_score:
                best_score = score
                best_idx = i
                best_step = row["step"]
                row["reduced_score"] = score
                best_row = row
        dfs.append(best_row)
        used[best_idx] = True
    
        range_ = score2range(best_score) 
        for r in range(1, int(range_)):
            reduce = range2score(range_ - r) + 1
            reduce_rate[best_step + r] = max(reduce_rate[best_step + r], reduce)
            if best_step - r >= 0:
                reduce_rate[best_step - r] = max(reduce_rate[best_step - r], reduce)
df = pd.DataFrame(dfs)
df.shape

100%|██████████| 269/269 [34:56<00:00,  7.80s/it]


(157495, 7)

In [39]:
sub = df.copy()
sub["score"] = sub["reduced_score"]

In [40]:
# スコア計算
labels = pd.read_csv(f"{CFG['dataset']['competition_dir']}/train_events.csv").dropna()
score, ap_table = compute_comptetition_metric(labels, sub)
print(f"score: {score:.4f}")
display(ap_table)

score: 0.7334


event   tolerance
onset   12           0.172485
        36           0.557710
        60           0.696708
        90           0.766346
        120          0.804185
        150          0.822615
        180          0.835692
        240          0.849712
        300          0.858579
        360          0.866315
wakeup  12           0.194300
        36           0.574497
        60           0.713173
        90           0.786630
        120          0.821101
        150          0.841289
        180          0.855762
        240          0.870829
        300          0.886042
        360          0.893476
dtype: float64